In [1]:
import pandas as pd

df = pd.read_csv("Behavioral_Risk_Factor_Surveillance_System__BRFSS__Prevalence_Data__2011_to_present_.csv")

df.head()

,Year,Locationabbr,Locationdesc,Class,Topic,Question,Response,Break_Out,Break_Out_Category,Sample_Size,...,Data_Value_Footnote,DataSource,ClassId,TopicId,LocationID,BreakoutID,BreakOutCategoryID,QuestionID,ResponseID,GeoLocation
0,2012,GU,Guam,Demographics,Age,What is your age?,55-64 years,Overall,Overall,296,...,NaN,BRFSS,CLASS05,Topic02,66,BO1,CAT1,AGE,RESP014,"(13.444304, 144.793731)"
1,2012,GU,Guam,Demographics,Age,What is your age?,35-44 years,Female,Gender,272,...,NaN,BRFSS,CLASS05,Topic02,66,SEX2,CAT2,AGE,RESP012,"(13.444304, 144.793731)"
2,2020,AL,Alabama,Chronic Health Indicators,Depression,Ever told you that you have a form of depression?,Yes,College graduate,Education Attained,356,...,NaN,BRFSS,CLASS03,TOPIC17,1,EDUCA4,CAT5,ADDEPEV3,RESP046,"(32.84057112200048, -86.63186076199969)"
3,2020,AL,Alabama,Chronic Health Indicators,Diabetes,Have you ever been told by a doctor that you h...,"Yes, pregnancy-related","White, non-Hispanic",Race/Ethnicity,32,...,NaN,BRFSS,CLASS03,TOPIC18,1,RACE01,CAT4,DIABETE4,RESP051,"(32.84057112200048, -86.63186076199969)"
4,2020,AL,Alabama,Chronic Health Indicators,Diabetes,Have you ever been told by a doctor that you h...,"No, pre-diabetes or borderline diabetes",Overall,Overall,92,...,NaN,BRFSS,CLASS03,TOPIC18,1,BO1,CAT1,DIABETE4,RESP052,"(32.84057112200048, -86.63186076199969)"


In [26]:
sampledf = df.sample(frac=0.0001)
depressiondf = df[(df['Question'] == "Ever told you that you have a form of depression?") & (df['Break_Out'] == "Overall")][["Year", "Locationabbr", "Response", "Sample_Size"]]
# depressiondf


In [39]:
# newdf = depressiondf.groupby("Year")
import json
def getDepressionTimeline(df):
    depressiondf = df[(df['Question'] == "Ever told you that you have a form of depression?") & (df['Break_Out'] == "Overall")][["Year", "Locationabbr", "Response", "Sample_Size"]]
    newdf = depressiondf.groupby("Year")
    dict = {"yearList": [], "percentageList": []}
    for name, group in newdf:
        dict["yearList"].append(name)
        dict["percentageList"].append(group[group['Response'] == "Yes"]['Sample_Size'].sum() / group['Sample_Size'].sum())

    return dict

json.dumps(getDepressionTimeline(df))

'{"yearList": [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], "percentageList": [0.1812508562680809, 0.18521903202113918, 0.1957961416642672, 0.19089408195283022, 0.19089444510180492, 0.17601937077992985, 0.19931488507029682, 0.18809343935925196, 0.1904975864330869, 0.18978010963273284, 0.19593154621409684]}'

In [40]:
df['Question'].unique()

array(['What is your age?',
       'Ever told you that you have a form of depression?',
       'Have you ever been told by a doctor that you have diabetes?',
       'How is your general health?', 'What is your marital status?',
       'Adults aged 50-75 who have had a blood stool test within the past year (variable calculated from one or more BRFSS questions)',
       'What is your employment status?',
       'About how long has it been since you last visited a doctor for a routine checkup?',
       'Since 2005, have you had a tetanus shot?',
       'Ever told you have vision impairment?',
       'What is your race/ethnicity?', 'Gender of respondent',
       'Adults who reported being deaf',
       'Do you have any kind of health care coverage?',
       'Do you currently use chewing tobacco, snuff, or snus?',
       'Four Level Smoking Status (variable calculated from one or more BRFSS questions)',
       'Have you ever been tested for HIV?',
       'Always or nearly always wears a sea

In [42]:
def getTimeline(df, question):
    depressiondf = df[(df['Question'] == question) & (df['Break_Out'] == "Overall")][["Year", "Locationabbr", "Response", "Sample_Size"]]
    newdf = depressiondf.groupby("Year")
    dict = {"yearList": [], "percentageList": []}
    for name, group in newdf:
        dict["yearList"].append(name)
        dict["percentageList"].append(group[group['Response'] == "Yes"]['Sample_Size'].sum() / group['Sample_Size'].sum())

    return dict

getTimeline(df, "Adults who are current smokers (variable calculated from one or more BRFSS questions)")

{'yearList': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021],
 'percentageList': [0.1682776827530438,
  0.1634486279186489,
  0.16094524597248766,
  0.14980358394504406,
  0.14559543422537138,
  0.14795204474109946,
  0.14732329084588644,
  0.14584580873896225,
  0.14147127725607192,
  0.13801283767116457,
  0.1303001239342979]}

In [64]:
def getGender(df, question):
    depressiondf = df[(df['Question'] == question) & ((df['Break_Out'] == "Female") | (df['Break_Out'] == "Male"))][["Year", "Break_Out", "Response", "Sample_Size"]]
    newdf = depressiondf.groupby(["Year", "Break_Out"])
    dict = {"yearList": [], "fpercentageList": [], "mpercentageList": [], "genderList": []}
    # gender = newdf.groupby("Break_Out")
    for (year, gender), group in newdf:
        dict["yearList"].append(year)
        dict["genderList"].append(gender)
        if gender == "Male":
            dict["mpercentageList"].append(group[group['Response'] == "Yes"]['Sample_Size'].sum() / group['Sample_Size'].sum())
        else:
            dict["fpercentageList"].append(group[group['Response'] == "Yes"]['Sample_Size'].sum() / group['Sample_Size'].sum())

    return dict

info = getGender(df, "Ever told you that you have a form of depression?")
print(info['fpercentageList'])
print(info['mpercentageList'])

[0.2117042792402772, 0.21737792694183428, 0.2327051633910116, 0.22765162296736322, 0.22788502145092732, 0.21158139721227706, 0.24035102462214433, 0.2282707662872224, 0.23248539349570604, 0.2345536175234162, 0.24349482153556448]
[0.1336917090570091, 0.13722798321133078, 0.14213778668092059, 0.13877504257302511, 0.14013050658719356, 0.12909504658585108, 0.14706864113612916, 0.13914557861814045, 0.13958418349771318, 0.1363231298281678, 0.14074443500760433]
